Load diabetes data set into data frame

In [ ]:
import pandas as pd

from sklearn import datasets

#Load features of diabetes dataset from sklearn datsets into a dataframe
diabetes= datasets.load_diabetes(return_X_y=False)
df=pd.DataFrame(diabetes.data,columns=diabetes.feature_names)
df['label']=diabetes.target
df

Split data into training and test data

In [ ]:
num_training_examples=300
num_features=10

train_features=df.iloc[:num_training_examples,:num_features]
train_labels=df.iloc[:num_training_examples,num_features]
test_features=df.iloc[num_training_examples:,:num_features]
test_labels=df.iloc[num_training_examples:,num_features]

train_labels

Fit regression model to training data and print regression coefficiants

In [ ]:
#Linear Regression
from sklearn import linear_model
reg = linear_model.LinearRegression()

#Fit regression model to training data
reg.fit(train_features,train_labels)

#Print regression coefficiants
print("Regression coefficients: ",reg.coef_)

Predict a single value and calculate deviation to actual value

In [ ]:
print("Predict label for following example: ",test_features.iloc[0,:])

#Predict a single value
predicted_value=reg.predict([test_features.iloc[0,:]])[0]

print("\nPredicted label: ",predicted_value)
print("Actual label:", test_labels.iloc[0])
print("Deviation predicted from actual value: ",predicted_value-test_labels.iloc[0])

Predict values for all test examples

In [ ]:
#Predict all test examples
ypred = reg.predict(test_features)
ypred

# Evaluate model

Calculate MAE and R*R

In [ ]:
from sklearn.metrics import mean_absolute_error

#Return Mean Absolute Error
mae = mean_absolute_error(test_labels, ypred)
print('MAE: %.3f' % mae)

#Return the coefficient of determination R^2 of the prediction.
print("R^2 value of the model: ",reg.score(test_features,test_labels))

Estimate p-values

In [51]:
####################### Calculate p-values with statsmodels package #######################
import statsmodels.api as sm
from scipy import stats

diabetes = datasets.load_diabetes()
X = train_features
y = train_labels

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
####################### Alternative: Calculate p-values without statsmodels package #######################

import numpy as np
from scipy import stats

X= test_features
y= test_labels

# Predictions and residuals
y_pred = ypred
residuals = y - y_pred

# Degrees of freedom
n, k = X.shape
df = n - k

# Residual variance
residual_var = np.sum(residuals ** 2) / df

# Add intercept term if model includes intercept
if reg.fit_intercept:
    X_design = np.column_stack((np.ones(n), X))
    coef = np.insert(reg.coef_, 0, reg.intercept_)
    feature_names = ['Intercept'] + list(X.columns)
else:
    X_design = X
    coef = reg.coef_

# Variance-Covariance matrix of beta
XTX_inv = np.linalg.inv(X.T @ X)
var_beta = residual_var * XTX_inv

# Standard errors of coefficients
se_beta = np.sqrt(np.diag(var_beta))

# t-statistics
t_stats = coef / se_beta

# p-values from t-distribution (two-tailed)
p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), df))

# Display the results
#drop first feature name (intercept)
results = pd.DataFrame({
    'Feature': feature_names[1:],
    'Coefficient': coef,
    'Standard Error': se_beta,
    't-Statistic': t_stats,
    'p-Value': p_values
})

results


Feature  Coefficient  Standard Error  t-Statistic   p-Value
0      age   152.347796      135.380821     1.125328  0.262508
1      sex   -16.573386      128.481793    -0.128994  0.897560
2      bmi  -254.663438      135.715886    -1.876445  0.062819
3       bp   560.989461      131.746072     4.258111  0.000039
4       s1   278.909652      833.805292     0.334502  0.738536
5       s2  -393.455577      647.745290    -0.607423  0.544622
6       s3    97.088553      444.666484     0.218340  0.827504
7       s4   -18.984276      345.608078    -0.054930  0.956278
8       s5   169.466162      375.263685     0.451592  0.652310
9       s6   632.968471      138.483536     4.570713  0.000011
10   const   114.218330        5.471647    20.874581  0.000000

Check normality of error terms

In [ ]:
deviation = ypred-test_labels
deviation.hist(bins=20)

Create polynomial regression model for comparison

In [ ]:
#Compare to polynomial regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

#Initialize and fit polynomial regression
degree=2
poly_reg=make_pipeline(PolynomialFeatures(degree),linear_model.LinearRegression())
poly_reg.fit(train_features,train_labels)

#Predict all test examples
ypred = poly_reg.predict(test_features)

mae = mean_absolute_error(test_labels, ypred)
print('MAE: %.3f' % mae)

Create KNN regression model for comparison

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

#Initialize and fit KNN regression
KNN_reg = KNeighborsRegressor(n_neighbors=10)
KNN_reg.fit(train_features,train_labels)

#Predict all test examples
ypred = KNN_reg.predict(test_features)

mae = mean_absolute_error(test_labels, ypred)
print('MAE: %.3f' % mae)

Create regression tree for comparison

In [ ]:
from sklearn.tree import DecisionTreeRegressor

#Initialize and fit regression tree
tree_reg = DecisionTreeRegressor(random_state=0,max_depth=3)
tree_reg.fit(train_features,train_labels)

#Predict all test examples
ypred = tree_reg.predict(test_features)

mae = mean_absolute_error(test_labels, ypred)
print('MAE: %.3f' % mae)